In [5]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import urllib.request
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
%matplotlib inline

In [6]:
'''NFL FASTR Documentation:
https://www.rdocumentation.org/packages/nflscrapR/versions/1.8.1/topics/scrape_json_play_by_play'''

'NFL FASTR Documentation:\nhttps://www.rdocumentation.org/packages/nflscrapR/versions/1.8.1/topics/scrape_json_play_by_play'

In [7]:
'''Procure Data'''

URL = 'https://github.com/guga31bb/nflfastR-data/blob/master/'
Unzip = '.csv.gz?raw=True'

In [8]:
'''Pull data'''

def pull_PBP_data(YEAR):
    
    data = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(YEAR) + Unzip,
                         compression='gzip', low_memory=False)
    
    return data

In [9]:
'''Data frames from last 3 years'''

data_2020 = pd.DataFrame(pull_PBP_data(2020))
data_2019 = pd.DataFrame(pull_PBP_data(2019))
data_2018 = pd.DataFrame(pull_PBP_data(2018))


In [10]:
# Neutral Situation

def Neutral_Situation_Call(DF):
    neutral_situation = DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8 & play_type != 'no_play'")
    return neutral_situation

neutral_situation = Neutral_Situation_Call(data_2020)

In [22]:
# Team List
Team_List = ['ARI',
'ATL',
'BAL',
'BUF',
'CAR',
'CHI',
'CIN',
'CLE',
'DAL',
'DEN',
'DET',
'GB',
'HOU',
'IND',
'JAX',
'KC',
'LA',
'LAC',
'LV',
'MIA',
'MIN',
'NE',
'NO',
'NYG',
'NYJ',
'PHI',
'PIT',
'SEA',
'SF',
'TB',
'TEN',
'WAS']





In [35]:
#CHOOSE TEAM

def Select_Team():
    
    team = input('Input Team Name:')
    
    team = team.upper()

    if team in Team_List:
        return team
    else:
        print('Please input a valid team')

    return team

print(Select_Team())



CHI


In [36]:
## PLAYCALL FREQUENCY BY PLAY TYPE & DOWN

def PlayFreq_ByTeam_Down(DF, PlayType, Team):
 
    PT = str(PlayType)


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[neutral_situation['play_type'] == PT]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.play_type,Play_filter.run_location, Play_filter.run_gap], columns= [Play_filter.shotgun,Play_filter.down] ,values = Play_filter.play_id, aggfunc='count', normalize='columns')
    elif PlayType == 'pass':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.play_type,Play_filter.pass_location, Play_filter.pass_length], columns= [ Play_filter.shotgun, Play_filter.down] ,values = Play_filter.play_id, aggfunc='count', normalize='columns')
    else:
        print("invalid play type")


    return round(Tab*100,1)

PlayFreq_ByTeam_Down(data_2020,'pass',Team)

<ipython-input-36-d6ce4d753ee5>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


shotgun                                         0           1            
down                                          1.0   2.0   1.0   2.0   3.0
posteam play_type pass_location pass_length                              
CHI     pass      left          deep         14.0  11.1   0.0   6.2   3.9
                                short        30.0  33.3  32.7  34.4  21.1
                  middle        deep          2.0  18.5   1.9   3.1   6.6
                                short         8.0   3.7  28.8  15.6  26.3
                  right         deep          8.0   7.4   9.6   4.7   5.3
                                short        38.0  25.9  26.9  35.9  36.8

In [62]:
## PLAYCALL TYPE FREQUENCY BY DOWN

def PlayFreqType_ByTeam_Down(DF, Team):
 
  


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[(neutral_situation['play_type'] == 'pass') | (neutral_situation['play_type'] == 'run') ]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    
    Tab = pd.crosstab([Play_filter.posteam,  Play_filter.play_type], columns= [Play_filter.shotgun,Play_filter.down] ,values = Play_filter.play_id, aggfunc='count', normalize='columns')
    

    return round(Tab*100,1)

PlayFreqType_ByTeam_Down(data_2020,Team)


<ipython-input-62-c11c0a8a5dc3>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


shotgun               0                  1            
down                1.0   2.0    3.0   1.0   2.0   3.0
posteam play_type                                     
CHI     pass       37.7  34.2    0.0  67.1  68.6  82.8
        run        62.3  65.8  100.0  32.9  31.4  17.2

In [37]:
## EPA BY DOWN & FORMATION


def PlayEPA_ByTeam_Down(DF, PlayType, Team):
 
    PT = str(PlayType)


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[neutral_situation['play_type'] == PT]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'run_location', 'run_gap'], columns = ['shotgun','down'], values = ['epa', 'yards_gained' ],fill_value = '-')
    elif PlayType == 'pass':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'pass_location', 'pass_length'], columns = ['shotgun', 'down'], values = ['epa', 'yards_gained', 'air_yards'],fill_value = '-')
    else:
        print("invalid play type")


    return round(Tab,2)

PlayEPA_ByTeam_Down(data_2020,'run',Team)

<ipython-input-37-e26c5f8815fe>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


epa                                \
shotgun                                    0                 1               
down                                     1.0   2.0   3.0   1.0   2.0   3.0   
posteam play_type run_location run_gap                                       
CHI     run       left         end     -0.23 -0.34  1.18 -0.48  0.70  0.33   
                               guard   -0.53 -0.02 -1.19 -0.36 -0.40  0.49   
                               tackle  -0.12  0.17   NaN  1.38 -0.50   NaN   
                  right        end     -0.04  0.54  1.55 -0.07  0.39  0.86   
                               guard   -0.46  0.18  0.51 -0.45 -0.51 -0.44   
                               tackle  -0.01 -0.00   NaN  0.11 -0.22 -1.83   

                                       yards_gained                           \
shotgun                                           0                 1          
down                                            1.0   2.0  3.0    1.0    2.0   
posteam play_type run_location run_gap                                         
CHI     run       left         end             3.25  2.11  7.0   1.40  11.00   
                               guard           1.33  2.50  0.0   1.00   3.20   
                               tackle          4.18  4.40  NaN  22.00   3.00   
                  right        end             4.24  5.40  7.0   4.60   8.60   
                               guard           1.60  5.33  3.0   1.50   2.25   
                               tackle          3.96  4.09  NaN   5.33   3.67   

                                              
shotgun                                       
down                                     3.0  
posteam play_type run_location run_gap        
CHI     run       left         end      2.00  
                               guard    3.33  
                               tackle    NaN  
                  right        end      6.00  
                               guard    1.50  
                               tackle   0.00

In [79]:
# EPA by Team, Play-type and Down -- HI LEVEL


def EPA_ByTeam_Down(DF, Team):

    Team_table = neutral_situation[neutral_situation['posteam']==Team]
    
    Table_2 = round(pd.pivot_table(Team_table, index= ['posteam','play_type', 'down'], columns = ['shotgun'], values = ['epa', 'yards_gained'], fill_value = '-'),2)

    return pd.DataFrame(Table_2) 

EPA_ByTeam_Down(data_2020,Team)



epa       yards_gained      
shotgun                    0     1            0     1
posteam play_type down                               
CHI     pass      1.0   0.11  0.01         7.13  6.09
                  2.0   0.50 -0.10        10.63  5.16
                  3.0    NaN -0.39          NaN  4.10
        run       1.0  -0.16  0.09         3.43  6.56
                  2.0   0.05  0.03         4.06  6.41
                  3.0   0.09 -0.06         3.00  2.59

In [81]:
## EPA BY DOWN & FORMATION


def PlayEPA_ByTeam_Down_Player(DF, PlayType, Team):
 
    PT = str(PlayType)


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[neutral_situation['play_type'] == PT]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'run_location', 'run_gap'], columns = ['shotgun','down', 'rusher'], values = ['epa', 'yards_gained' ], aggfunc='mean', fill_value = '-')
    elif PlayType == 'pass':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'pass_location', 'pass_length'], columns = ['shotgun', 'down'], values = ['epa', 'yards_gained', 'air_yards'], aggfunc='mean', fill_value = '-')
    else:
        print("invalid play type")


    return round(Tab,2)

PlayEPA_ByTeam_Down_Player(data_2020,'run',Team)

<ipython-input-81-0dee506e202e>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


epa                       \
shotgun                                        0                        
down                                         1.0                        
rusher                                  A.Miller  A.Pierce A.Robinson   
posteam play_type run_location run_gap                                  
CHI     run       left         end     -0.285854         -  -0.903545   
                               guard           -         -          -   
                               tackle          -         -          -   
                  right        end             - -0.086524          -   
                               guard           -         -          -   
                               tackle          -  0.244609          -   

                                                                        \
shotgun                                                                  
down                                                                     
rusher                                    C.Kmet C.Patterson  D.Carter   
posteam play_type run_location run_gap                                   
CHI     run       left         end             -       -0.25         -   
                               guard           -       -0.44         -   
                               tackle          -       -0.04         -   
                  right        end     -1.022734        0.30 -0.132055   
                               guard           -       -0.34         -   
                               tackle          -       -0.31         -   

                                                                         \
shotgun                                                                   
down                                                                      
rusher                                 D.Montgomery  D.Mooney   T.Cohen   
posteam play_type run_location run_gap                                    
CHI     run       left         end             0.06         -         -   
                               guard          -0.55         -         -   
                               tackle         -0.18         -         -   
                  right        end            -0.33  0.277179         -   
                               guard          -0.48         -         -   
                               tackle          0.01         - -0.017823   

                                                  ... yards_gained  \
shotgun                                           ...            1   
down                                         2.0  ...          1.0   
rusher                                  A.Miller  ...   M.Trubisky   
posteam play_type run_location run_gap            ...                
CHI     run       left         end      0.896283  ...          1.0   
                               guard           -  ...            -   
                               tackle          -  ...            -   
                  right        end             -  ...          8.0   
                               guard           -  ...            -   
                               tackle          -  ...            -   

                                                                         \
shotgun                                                                   
down                                           2.0                        
rusher                                 C.Patterson D.Montgomery N.Foles   
posteam play_type run_location run_gap                                    
CHI     run       left         end               -         5.00       -   
                               guard           0.0         4.00       -   
                               tackle            -         3.00       -   
                  right        end            -1.0        10.67       -   
                               guard             -         2.25       -   
                               tackle            -         5.00     0.0   

   

In [84]:
def PlayFreqType_ByTeam_Down_Player(DF,PlayType,Team):
 
  


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[(neutral_situation['play_type'] == PlayType)]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.down,Play_filter.rusher], columns= [Play_filter.run_location,Play_filter.run_gap] ,values = Play_filter.play_id, aggfunc='count', normalize='index')
    elif PlayType == 'pass':
        Tab = pd.crosstab([Play_filter.down, Play_filter.receiver ], columns= [Play_filter.pass_location,Play_filter.pass_length] ,values = Play_filter.play_id, aggfunc='count', normalize='index')
    else: 
        print("invalid play type")



    return round(Tab*100,1)

PlayFreqType_ByTeam_Down_Player(data_2020,'run',Team)

<ipython-input-84-6fef571926b0>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


run_location                left                right              
run_gap                      end  guard tackle    end  guard tackle
posteam down rusher                                                
CHI     1.0  A.Miller      100.0    0.0    0.0    0.0    0.0    0.0
             A.Pierce        0.0    0.0    0.0   50.0    0.0   50.0
             A.Robinson    100.0    0.0    0.0    0.0    0.0    0.0
             C.Kmet          0.0    0.0    0.0  100.0    0.0    0.0
             C.Patterson    20.0    8.0   32.0   28.0    4.0    8.0
             D.Carter        0.0    0.0    0.0  100.0    0.0    0.0
             D.Montgomery    9.1   10.6   21.2   10.6   16.7   31.8
             D.Mooney        0.0    0.0    0.0  100.0    0.0    0.0
             M.Trubisky     66.7    0.0    0.0   33.3    0.0    0.0
             T.Cohen         0.0    0.0    0.0    0.0    0.0  100.0
        2.0  A.Miller      100.0    0.0    0.0    0.0    0.0    0.0
             A.Pierce      100.0    0.0    0.0    0.0    0.0    0.0
             C.Patterson    35.7   14.3   14.3   14.3   14.3    7.1
             D.Montgomery    4.1   10.2   22.4   10.2   26.5   26.5
             D.Mooney      100.0    0.0    0.0    0.0    0.0    0.0
             M.Trubisky      0.0    0.0    0.0    0.0  100.0    0.0
             N.Foles         0.0    0.0    0.0    0.0    0.0  100.0
             T.Cohen        50.0    0.0    0.0    0.0    0.0   50.0
        3.0  A.Pierce        0.0    0.0    0.0    0.0  100.0    0.0
             C.Patterson     0.0    0.0    0.0    0.0  100.0    0.0
             D.Montgomery    7.7   46.2    0.0    7.7   23.1   15.4
             M.Trubisky    100.0    0.0    0.0    0.0    0.0    0.0
             T.Cohen         0.0  100.0    0.0    0.0    0.0    0.0